In [1]:
# NOTEBOOK 5: RANDOM FOREST CON FEATURE SELECTION
# Optimizado para Google Colab

# IMPORTACIÓN DE LIBRERÍAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
import gc  # Para manejo explícito de memoria

warnings.filterwarnings('ignore')

In [2]:
print("="*80)
print("NOTEBOOK 5: RANDOM FOREST CON FEATURE SELECTION")
print("="*80)

# CONFIGURACIÓN DE PARÁMETROS PARA OPTIMIZACIÓN DE MEMORIA
# Parámetros para limitar uso de memoria
MAX_FEATURES_INITIAL = 100  # Límite inicial de características
MAX_FEATURES_FINAL = 30     # Máximo de características para modelo final
SAMPLE_SIZE_LIMIT = 50000   # Límite de muestras para evitar sobrecarga
CHUNK_SIZE = 10000          # Tamaño de chunks para procesamiento

print(f"Configuración de memoria:")
print(f"- Máximo de características iniciales: {MAX_FEATURES_INITIAL}")
print(f"- Máximo de características finales: {MAX_FEATURES_FINAL}")
print(f"- Límite de muestras: {SAMPLE_SIZE_LIMIT}")

NOTEBOOK 5: RANDOM FOREST CON FEATURE SELECTION
Configuración de memoria:
- Máximo de características iniciales: 100
- Máximo de características finales: 30
- Límite de muestras: 50000


In [3]:
# CARGA Y PREPROCESAMIENTO DE DATOS (OPTIMIZADO)
print("\n1. CARGA DE DATOS OPTIMIZADA")
print("-" * 40)

# Cargar datos con optimización de memoria
try:
    # Leer solo las columnas necesarias y con tipos optimizados
    df_train = pd.read_csv('train.csv', low_memory=False)

    # Si el dataset es muy grande, tomar una muestra
    if len(df_train) > SAMPLE_SIZE_LIMIT:
        print(f"Dataset grande detectado ({len(df_train)} filas)")
        print(f"Tomando muestra aleatoria de {SAMPLE_SIZE_LIMIT} filas")
        df_train = df_train.sample(n=SAMPLE_SIZE_LIMIT, random_state=42)
        gc.collect()  # Liberar memoria

    print(f"✓ Datos de entrenamiento cargados: {df_train.shape[0]} filas x {df_train.shape[1]} columnas")

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Asegúrate de tener el archivo 'train.csv' en el directorio.")
    raise

# Usar solo datos de entrenamiento
df = df_train.copy()
del df_train  # Liberar memoria inmediatamente
gc.collect()

print(f"Trabajando con datos: {df.shape}")


1. CARGA DE DATOS OPTIMIZADA
----------------------------------------
Dataset grande detectado (80827 filas)
Tomando muestra aleatoria de 50000 filas
✓ Datos de entrenamiento cargados: 50000 filas x 21 columnas
Trabajando con datos: (50000, 21)


In [4]:
print("\n2. ANÁLISIS INICIAL OPTIMIZADO")
print("-" * 40)

# Mostrar información básica de forma eficiente
print(f"Forma del dataset: {df.shape}")
print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Analizar valores nulos de forma eficiente
nulos = df.isnull().sum()
nulos_pct = (nulos / len(df) * 100).round(2)
nulos_info = pd.DataFrame({'Nulos': nulos, 'Porcentaje': nulos_pct})
nulos_info = nulos_info[nulos_info['Nulos'] > 0]

if len(nulos_info) > 0:
    print(f"\nColumnas con valores nulos:")
    print(nulos_info.head(10))  # Mostrar solo top 10
else:
    print("No hay valores nulos")


2. ANÁLISIS INICIAL OPTIMIZADO
----------------------------------------
Forma del dataset: (50000, 21)
Memoria utilizada: 52.70 MB

Columnas con valores nulos:
                                Nulos  Porcentaje
ESTU_VALORMATRICULAUNIVERSIDAD    456        0.91
ESTU_HORASSEMANATRABAJA          2203        4.41
FAMI_ESTRATOVIVIENDA             2320        4.64
FAMI_TIENEINTERNET               1897        3.79
FAMI_EDUCACIONPADRE              1658        3.32
FAMI_TIENELAVADORA               2840        5.68
FAMI_TIENEAUTOMOVIL              3109        6.22
ESTU_PAGOMATRICULAPROPIO          469        0.94
FAMI_TIENECOMPUTADOR             2735        5.47
FAMI_TIENEINTERNET.1             1897        3.79


In [5]:
print("\n3. PREPROCESAMIENTO OPTIMIZADO")
print("-" * 40)

# Identificar tipos de columnas de forma eficiente
columnas_categoricas = df.select_dtypes(include=['object']).columns.tolist()
columnas_numericas = df.select_dtypes(include=['number']).columns.tolist()

print(f"Columnas categóricas: {len(columnas_categoricas)}")
print(f"Columnas numéricas: {len(columnas_numericas)}")

# Configurar columnas para procesamiento
target_column = 'RENDIMIENTO_GLOBAL'
columns_to_drop = ['ID'] if 'ID' in df.columns else []

# Verificar que la columna objetivo existe
if target_column not in df.columns:
    possible_targets = [col for col in df.columns if 'RENDIMIENTO' in col.upper() or 'TARGET' in col.upper()]
    if possible_targets:
        target_column = possible_targets[0]
        print(f"Usando columna: {target_column}")
    else:
        raise ValueError("No se encontró columna objetivo")

# Separar características
cat_columns = [col for col in columnas_categoricas if col not in columns_to_drop + [target_column]]
num_columns = [col for col in columnas_numericas if col not in columns_to_drop + [target_column]]

print(f"Procesando {len(cat_columns)} categóricas y {len(num_columns)} numéricas")

# Procesamiento optimizado de datos
print("\nProcesando datos de forma eficiente...")

# Imputar valores nulos en columnas numéricas
for col in num_columns:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# Imputar valores nulos en columnas categóricas
for col in cat_columns:
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()[0] if len(df[col].mode()) > 0 else 'UNKNOWN'
        df[col].fillna(mode_val, inplace=True)

# One-hot encoding optimizado (solo para categorías con pocas clases)
if cat_columns:
    print("Aplicando One-Hot Encoding optimizado...")
    cat_to_encode = []

    for col in cat_columns:
        unique_vals = df[col].nunique()
        if unique_vals <= 20:  # Solo encodear si tiene pocas categorías
            cat_to_encode.append(col)
        else:
            print(f"Columna {col} omitida (demasiadas categorías: {unique_vals})")

    if cat_to_encode:
        df = pd.get_dummies(df, columns=cat_to_encode, drop_first=True)
        print(f"Encoding aplicado a {len(cat_to_encode)} columnas")

    # Eliminar columnas categóricas no procesadas
    remaining_cat = [col for col in cat_columns if col not in cat_to_encode]
    if remaining_cat:
        df = df.drop(columns=remaining_cat)
        print(f"Eliminadas {len(remaining_cat)} columnas categóricas complejas")

# Normalización optimizada
if num_columns:
    print("Normalizando variables numéricas...")
    scaler = StandardScaler()
    df[num_columns] = scaler.fit_transform(df[num_columns])

print(f"Dimensiones finales: {df.shape}")
gc.collect()  # Liberar memoria

# Verificar variable objetivo
print(f"Variable objetivo: {target_column}")
print(f"  Distribución: {df[target_column].value_counts().to_dict()}")


3. PREPROCESAMIENTO OPTIMIZADO
----------------------------------------
Columnas categóricas: 15
Columnas numéricas: 6
Procesando 14 categóricas y 5 numéricas

Procesando datos de forma eficiente...
Aplicando One-Hot Encoding optimizado...
Columna ESTU_PRGM_ACADEMICO omitida (demasiadas categorías: 842)
Columna ESTU_PRGM_DEPARTAMENTO omitida (demasiadas categorías: 30)
Encoding aplicado a 12 columnas
Eliminadas 2 columnas categóricas complejas
Normalizando variables numéricas...
Dimensiones finales: (50000, 53)
Variable objetivo: RENDIMIENTO_GLOBAL
  Distribución: {'alto': 12568, 'medio-bajo': 12550, 'bajo': 12477, 'medio-alto': 12405}


In [6]:
# ANÁLISIS EXPLORATORIO OPTIMIZADO
print("\n4. ANÁLISIS EXPLORATORIO OPTIMIZADO")
print("-" * 40)

# Distribución de la variable objetivo (sin gráfico para ahorrar memoria)
target_dist = df[target_column].value_counts().sort_index()
print("Distribución de Rendimiento Global:")
for idx, count in target_dist.items():
    print(f"{idx}: {count} ({count/len(df)*100:.1f}%)")

# Preparar datos para el modelo
X = df.drop(columns=[target_column])
y = df[target_column]

# Limitar número de características si es muy alto
if X.shape[1] > MAX_FEATURES_INITIAL:
    print(f"Demasiadas características ({X.shape[1]})")
    print(f"Seleccionando las {MAX_FEATURES_INITIAL} más importantes...")

    # Selección rápida basada en varianza
    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold(threshold=0.01)
    X_reduced = selector.fit_transform(X)
    feature_names = X.columns[selector.get_support()]

    if len(feature_names) > MAX_FEATURES_INITIAL:
        # Si aún hay muchas, tomar las primeras MAX_FEATURES_INITIAL
        feature_names = feature_names[:MAX_FEATURES_INITIAL]
        X_reduced = X_reduced[:, :MAX_FEATURES_INITIAL]

    X = pd.DataFrame(X_reduced, columns=feature_names, index=X.index)
    print(f"Características reducidas a: {X.shape[1]}")

# Codificar variable objetivo
le = LabelEncoder()
if y.dtype == 'object':
    y_encoded = le.fit_transform(y)
    print(f"Variable objetivo codificada")
else:
    y_encoded = y.values

print(f"Datos preparados: {X.shape[0]} muestras, {X.shape[1]} características")


4. ANÁLISIS EXPLORATORIO OPTIMIZADO
----------------------------------------
Distribución de Rendimiento Global:
alto: 12568 (25.1%)
bajo: 12477 (25.0%)
medio-alto: 12405 (24.8%)
medio-bajo: 12550 (25.1%)
Variable objetivo codificada
Datos preparados: 50000 muestras, 52 características


In [7]:
# FEATURE SELECTION OPTIMIZADO
print("\n5. SELECCIÓN DE CARACTERÍSTICAS OPTIMIZADA")
print("-" * 40)

# Método 1: Selección univariada (más rápida)
k_features = min(MAX_FEATURES_FINAL, X.shape[1])
print(f"Seleccionando {k_features} mejores características...")

selector_univariate = SelectKBest(score_func=f_classif, k=k_features)
X_selected = selector_univariate.fit_transform(X, y_encoded)
selected_features = X.columns[selector_univariate.get_support()]

print(f"Características seleccionadas: {len(selected_features)}")

# Mostrar las características más importantes
feature_scores = pd.DataFrame({
    'feature': selected_features,
    'score': selector_univariate.scores_[selector_univariate.get_support()]
}).sort_values('score', ascending=False)

print("\nTop 10 características más importantes:")
print(feature_scores.head(10))

# Liberar memoria
del X, selector_univariate
gc.collect()


5. SELECCIÓN DE CARACTERÍSTICAS OPTIMIZADA
----------------------------------------
Seleccionando 30 mejores características...
Características seleccionadas: 30

Top 10 características más importantes:
                                             feature        score
8   ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones  1770.524861
1                                             coef_1  1158.515394
22                       ESTU_PAGOMATRICULAPROPIO_Si   686.093028
2                                             coef_2   636.091539
17                     FAMI_EDUCACIONPADRE_Postgrado   598.611008
21                            FAMI_TIENEAUTOMOVIL_Si   527.494900
26                     FAMI_EDUCACIONMADRE_Postgrado   508.984996
28           FAMI_EDUCACIONMADRE_Primaria incompleta   443.126256
12                    FAMI_ESTRATOVIVIENDA_Estrato 4   372.063437
19           FAMI_EDUCACIONPADRE_Primaria incompleta   364.210212


0

In [8]:
# MODELO RANDOM FOREST OPTIMIZADO
print("\n6. ENTRENAMIENTO DEL MODELO OPTIMIZADO")
print("-" * 40)

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"División - Entrenamiento: {X_train.shape[0]}, Prueba: {X_test.shape[0]}")

# Modelo Random Forest optimizado para memoria
rf = RandomForestClassifier(
    n_estimators=100,  # Reducido para ahorrar memoria
    max_depth=15,      # Limitado para evitar overfitting y ahorrar memoria
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',  # Usar sqrt para reducir memoria
    random_state=42,
    n_jobs=2  # Limitado para Colab
)

print("Entrenando Random Forest...")
rf.fit(X_train, y_train)

# Predicciones
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.4f}")


6. ENTRENAMIENTO DEL MODELO OPTIMIZADO
----------------------------------------
División - Entrenamiento: 40000, Prueba: 10000
Entrenando Random Forest...
Precisión del modelo: 0.3741


In [9]:
# OPTIMIZACIÓN LIGERA DE HIPERPARÁMETROS
print("\n7. OPTIMIZACIÓN LIGERA DE HIPERPARÁMETROS")
print("-" * 40)

# Grid Search reducido para ahorrar memoria y tiempo
param_grid_light = {
    'n_estimators': [50, 100],
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10]
}

print("Realizando Grid Search optimizado...")
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=2, max_features='sqrt'),
    param_grid_light,
    cv=3,  # Reducido de 5 a 3 para ahorrar tiempo
    scoring='accuracy',
    n_jobs=1,  # Sin paralelización para evitar sobrecarga
    verbose=0
)

rf_grid.fit(X_train, y_train)

print(f"Mejores parámetros: {rf_grid.best_params_}")
print(f"Mejor score CV: {rf_grid.best_score_:.4f}")


7. OPTIMIZACIÓN LIGERA DE HIPERPARÁMETROS
----------------------------------------
Realizando Grid Search optimizado...
Mejores parámetros: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Mejor score CV: 0.3743


In [10]:
# EVALUACIÓN FINAL
print("\n8. EVALUACIÓN FINAL")
print("-" * 40)

# Predicciones con el mejor modelo
best_rf = rf_grid.best_estimator_
y_pred_final = best_rf.predict(X_test)
accuracy_final = accuracy_score(y_test, y_pred_final)

print(f"Precisión final: {accuracy_final:.4f}")

# Reporte de clasificación
print("\nReporte de Clasificación:")
if y.dtype == 'object':
    target_names = le.classes_
    print(classification_report(y_test, y_pred_final, target_names=target_names))
else:
    print(classification_report(y_test, y_pred_final))

# Matriz de confusión (solo mostrar, no graficar)
cm = confusion_matrix(y_test, y_pred_final)
print(f"\nMatriz de Confusión:")
print(cm)

# Importancia de características
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 características más importantes:")
print(feature_importance.head(10))


8. EVALUACIÓN FINAL
----------------------------------------
Precisión final: 0.3767

Reporte de Clasificación:
              precision    recall  f1-score   support

        alto       0.47      0.54      0.50      2514
        bajo       0.39      0.51      0.44      2495
  medio-alto       0.29      0.22      0.25      2481
  medio-bajo       0.31      0.24      0.27      2510

    accuracy                           0.38     10000
   macro avg       0.36      0.38      0.36     10000
weighted avg       0.36      0.38      0.37     10000


Matriz de Confusión:
[[1354  371  476  313]
 [ 334 1266  396  499]
 [ 726  716  552  487]
 [ 484  932  499  595]]

Top 10 características más importantes:
                                             feature  importance
8   ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones    0.193166
1                                             coef_1    0.130537
2                                             coef_2    0.100664
4                                   

In [11]:
# VALIDACIÓN CRUZADA LIGERA
print("\n9. VALIDACIÓN CRUZADA")
print("-" * 40)

# Validación cruzada con menos folds para ahorrar tiempo
cv_scores = cross_val_score(best_rf, X_selected, y_encoded, cv=3, scoring='accuracy')

print(f"Scores de validación cruzada: {cv_scores.round(4)}")
print(f"Precisión promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")


9. VALIDACIÓN CRUZADA
----------------------------------------
Scores de validación cruzada: [0.3771 0.3737 0.3733]
Precisión promedio: 0.3747 (+/- 0.0034)


In [12]:
# GUARDAR RESULTADOS
print("\n10. GUARDADO DE RESULTADOS")
print("-" * 40)

# Información del modelo
model_info = {
    'model_type': 'Random Forest Optimizado para Colab',
    'n_features_selected': len(selected_features),
    'best_params': rf_grid.best_params_,
    'final_accuracy': accuracy_final,
    'cv_mean_accuracy': cv_scores.mean(),
    'cv_std_accuracy': cv_scores.std(),
    'dataset_shape': f"{len(y_encoded)} x {len(selected_features)}"
}

# Guardar resultados
with open('notebook1_optimized_results.txt', 'w', encoding='utf-8') as f:
    f.write("RESULTADOS RANDOM FOREST OPTIMIZADO PARA COLAB\n")
    f.write("="*50 + "\n\n")
    for key, value in model_info.items():
        f.write(f"{key}: {value}\n")
    f.write(f"\nTop 10 características importantes:\n")
    for idx, row in feature_importance.head(10).iterrows():
        f.write(f"- {row['feature']}: {row['importance']:.4f}\n")

print("Resultados guardados en 'notebook1_optimized_results.txt'")

# Limpiar memoria final
del X_train, X_test, X_selected
gc.collect()

print("\n" + "="*80)
print("NOTEBOOK 5 OPTIMIZADO COMPLETADO")
print("="*80)
print(f"Modelo final: Random Forest optimizado para Colab")
print(f"Características utilizadas: {len(selected_features)}")
print(f"Precisión final: {accuracy_final:.4f}")
print(f"Uso de memoria optimizado para Google Colab")
print("="*80)


10. GUARDADO DE RESULTADOS
----------------------------------------
Resultados guardados en 'notebook1_optimized_results.txt'

NOTEBOOK 5 OPTIMIZADO COMPLETADO
Modelo final: Random Forest optimizado para Colab
Características utilizadas: 30
Precisión final: 0.3767
Uso de memoria optimizado para Google Colab
